In [5]:
# le plus important

import pandas as pd
import re

print("="*60)
print("NORMALISATION DES TRAITEMENTS VIH – ABRÉVIATIONS")
print("="*60)

# ------------------------------------------------------------------
# 1. Charger les fichiers
# ------------------------------------------------------------------
df_arv = pd.read_excel("data/Med_hep.xlsx")
df_acronym = pd.read_excel("data/medicaments_acronym.xlsx")

print(f"→ {len(df_acronym)} abréviations dans le dictionnaire")
print(f"→ {len(df_arv)} lignes ARV à traiter")

# ------------------------------------------------------------------
# 2. Préparation du dictionnaire d'abréviations
# ------------------------------------------------------------------
df_acronym["Abbreviation"] = df_acronym["Abbreviation"].fillna("").astype(str).str.strip()
df_acronym["Definition"] = df_acronym["Definition"].fillna("").astype(str).str.strip()

# Créer un dictionnaire trié par longueur décroissante
# (pour traiter les abréviations longues en premier, ex: "ATV/c" avant "ATV")
acronym_dict = {}
for _, row in df_acronym.iterrows():
    abbrev = row["Abbreviation"].strip()
    definition = row["Definition"].strip()
    if abbrev and definition:
        acronym_dict[abbrev] = definition

# Trier par longueur décroissante
acronym_dict = dict(sorted(acronym_dict.items(), key=lambda x: len(x[0]), reverse=True))

print(f"✓ Dictionnaire chargé : {len(acronym_dict)} abréviations")

# ------------------------------------------------------------------
# 3. Fonction de remplacement des abréviations
# ------------------------------------------------------------------

def remplacer_abreviations(traitement):
    """
    Remplace les abréviations trouvées par leurs définitions.
    Garde le texte original pour ce qui n'est pas trouvé.
    """
    if pd.isna(traitement) or not str(traitement).strip():
        return traitement

    traitement_str = str(traitement).strip()

    # Séparer par "+" pour traiter chaque médicament individuellement
    items = [t.strip() for t in traitement_str.split("+")]

    resultats = []
    for item in items:
        if not item:
            continue

        item_original = item
        item_modifie = item

        # Essayer de remplacer chaque abréviation dans l'item
        for abbrev, definition in acronym_dict.items():
            # Chercher l'abréviation dans l'item (insensible à la casse)
            # Pattern: l'abréviation doit être un mot entier ou suivi d'un espace/chiffre
            pattern = r'\b' + re.escape(abbrev) + r'\b'

            if re.search(pattern, item_modifie, flags=re.IGNORECASE):
                # Remplacer en préservant la casse originale si possible
                item_modifie = re.sub(
                    pattern,
                    definition,
                    item_modifie,
                    flags=re.IGNORECASE
                )
                # On sort de la boucle après le premier remplacement réussi
                break

        resultats.append(item_modifie)

    return " + ".join(resultats)

# ------------------------------------------------------------------
# 4. Appliquer sur tout le DataFrame
# ------------------------------------------------------------------

print("\nTraitement en cours...")
df_arv["Med_Hep_normalise"] = df_arv["Med_Hep"].apply(remplacer_abreviations)

# Statistiques
nb_changes = (df_arv["Med_Hep"].astype(str) != df_arv["Med_Hep_normalise"].astype(str)).sum()
nb_total = len(df_arv)

print(f"✓ Normalisation terminée")
print(f"  → Total de lignes traitées : {nb_total}")
print(f"  → Lignes modifiées : {nb_changes}")
print(f"  → Lignes inchangées : {nb_total - nb_changes}")

# ------------------------------------------------------------------
# 5. Afficher quelques exemples
# ------------------------------------------------------------------
print("\n" + "="*60)
print("EXEMPLES DE TRANSFORMATION:")
print("="*60)

# Afficher les 15 premières lignes où il y a eu un changement
exemples = df_arv[df_arv["Med_Hep"].astype(str) != df_arv["Med_Hep_normalise"].astype(str)].head(15)
if len(exemples) > 0:
    for idx, row in exemples.iterrows():
        print(f"\nAvant : {row['Med_Hep']}")
        print(f"Après : {row['Med_Hep_normalise']}")
else:
    print("Aucune transformation effectuée.")

# Afficher quelques exemples spécifiques pour vérification
print("\n" + "="*60)
print("VÉRIFICATION DES CAS SPÉCIFIQUES:")
print("="*60)

# Chercher des exemples avec "ABT", "Videx", "Zerit"
test_cases = df_arv[df_arv["Med_Hep"].astype(str).str.contains("ABT|Videx|Zerit", case=False, na=False)].head(5)
for idx, row in test_cases.iterrows():
    print(f"\nAvant : {row['Med_Hep']}")
    print(f"Après : {row['Med_Hep_normalise']}")

# ------------------------------------------------------------------
# 6. Sauvegarde
# ------------------------------------------------------------------
df_arv.to_excel("data/out/Med_hep_abbr.xlsx", index=False)
print("\n" + "="*60)
print("Fichier créé : data/out/Med_Hep_abbr.xlsx")
print("="*60)

NORMALISATION DES TRAITEMENTS VIH – ABRÉVIATIONS
→ 323 abréviations dans le dictionnaire
→ 286 lignes ARV à traiter
✓ Dictionnaire chargé : 313 abréviations

Traitement en cours...
✓ Normalisation terminée
  → Total de lignes traitées : 286
  → Lignes modifiées : 4
  → Lignes inchangées : 282

EXEMPLES DE TRANSFORMATION:

Avant : Ribavirine (Copegus / Rébétol) + Sovaldi  + Zepatier
Après : Ribavirine (Copegus / Rébétol) + Sovaldi + Zepatier

Avant : Sovaldi  + Zepatier
Après : Sovaldi + Zepatier

Avant : Ribavirine (Copegus / Rébétol) + Sovaldi  + Viraferon Peg
Après : Ribavirine (Copegus / Rébétol) + Sovaldi + Viraferon Peg

Avant : Daklinza + Sovaldi  + Viekirax
Après : Daklinza + Sovaldi + Viekirax

VÉRIFICATION DES CAS SPÉCIFIQUES:

Avant : Exviera + Ombitasvir ( ABT-267 ) + Paritaprevir (ABT- 450/ r ) + Ribavirine (Copegus / Rébétol) + Sovaldi
Après : Exviera + Ombitasvir ( ABT-267 ) + Paritaprevir (ABT- 450/ r ) + Ribavirine (Copegus / Rébétol) + Sovaldi

Avant : Dasabuvir ( ABT-

In [6]:
import pandas as pd 
df = pd.read_excel("data/out/Med_hep_abbr.xlsx")

In [7]:
df


,Med_Hep,Med_Hep_normalise
0,Copegus + Pegasys,Copegus + Pegasys
1,Rébétol + Viraferon Peg,Rébétol + Viraferon Peg
2,Viraferon Peg,Viraferon Peg
3,Harvoni,Harvoni
4,Epclusa,Epclusa
...,...,...
281,Copegus + Interferon + Pegasys,Copegus + Interferon + Pegasys
282,Lamivudine (Epivir),Lamivudine (Epivir)
283,Incivo (Telaprevir) + Pegasys,Incivo (Telaprevir) + Pegasys
284,Emtricitabine (Emtriva) + Hepsera,Emtricitabine (Emtriva) + Hepsera


In [8]:
df = df.drop(columns=['Med_Hep'])          # Supprime la colonne ARV
df = df.rename(columns={'Med_Hep_normalise': 'Med_Hep'})   # Renomme ARV_normalise → ARV


In [9]:
df

,Med_Hep
0,Copegus + Pegasys
1,Rébétol + Viraferon Peg
2,Viraferon Peg
3,Harvoni
4,Epclusa
...,...
281,Copegus + Interferon + Pegasys
282,Lamivudine (Epivir)
283,Incivo (Telaprevir) + Pegasys
284,Emtricitabine (Emtriva) + Hepsera


In [10]:
df.to_excel("data/out/Med_Hep_abbr.xlsx", index=False)
